this notebook calculates the similarity scores between words in posts via word vectors from Gensim

In [46]:
import numpy as np
import pandas as pd
import gensim.downloader as api
import nltk
from nltk import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import RegexpTokenizer
from gensim.models import Word2Vec, KeyedVectors

Change the folder either to `instagram_data/` or to `twitter_data/`

In [47]:
folder  = 'twitter/' #twitter_data/ or instagram_data/

In [48]:
data = pd.read_csv(folder +'posts_processed.csv')
green = pd.read_csv(folder +'green_posts_processed.csv')

/var/folders/pv/j2wxhdx50y5fbwqrc2yxqwf00000gn/T/ipykernel_66234/1928277876.py:1: DtypeWarning: Columns (19) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv(folder +'posts_processed.csv')


Assign either `data` or `green` to the `posts` variable

In [49]:
posts = green #change to data for whole data, green for sustainable posts
posts

,platformId,platform,date,updated,type,post_text,postUrl,subscriberCount,score,media,...,username,actual,likes,comments,Branche,Branche-6,Sektor,B2B,B2C,green
0,355091021065555690_10145706,Instagram,2012-12-26 19:29:29,2022-06-23 22:21:38,photo,Pro Green X #PUMA tee @professorgreen #PUMAlife,https://www.instagram.com/p/TtiWb9iw7q/,0,-57.086404,"[{'type': 'photo', 'url': 'https://scontent-yy...",...,puma,"{'favoriteCount': 775, 'commentCount': 12}",775,12,mode,Textil,2.0,True,True,True
1,346237364735773953_10145706,Instagram,2012-12-14 14:18:51,2022-06-23 22:21:38,photo,A day in the life of #PUMA #pumalife - morning...,https://www.instagram.com/p/TOFQx9Cw0B/,0,-29.654785,"[{'type': 'photo', 'url': 'https://scontent-yy...",...,puma,"{'favoriteCount': 1486, 'commentCount': 29}",1486,29,mode,Textil,2.0,True,True,True
2,342685722404917046_10145706,Instagram,2012-12-09 16:42:23,2022-06-23 22:21:38,photo,"#PUMA, always looking good our friend @profess...",https://www.instagram.com/p/TBdtlRiw82/,0,-79.799290,"[{'type': 'photo', 'url': 'https://scontent-yy...",...,puma,"{'favoriteCount': 557, 'commentCount': 6}",557,6,mode,Textil,2.0,True,True,True
3,333882232069275743_25749975,Instagram,2012-11-27 13:11:25,2022-05-27 23:50:24,photo,Mercedes-Benz SLS AMG Coupé Black Series! #mer...,https://www.instagram.com/p/SiMB7_TcBf/,0,-27.810777,"[{'type': 'photo', 'url': 'https://scontent-se...",...,mercedesbenz,"{'favoriteCount': 3219, 'commentCount': 10}",3219,10,automobil,Automobil,2.0,True,True,True
4,329423030961684553_25749975,Instagram,2012-11-21 09:31:47,2022-05-27 23:50:24,photo,Say hello to the newest @mercedesamg powerhous...,https://www.instagram.com/p/SSWIBVTcBJ/,0,-22.427822,"[{'type': 'photo', 'url': 'https://scontent-se...",...,mercedesbenz,"{'favoriteCount': 3979, 'commentCount': 25}",3979,25,automobil,Automobil,2.0,True,True,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9333,2742155642375737703_43109246,Instagram,2022-01-02 08:00:47,2022-07-07 23:43:38,album,A powerful start to the day.\nThe BMW M5 CS.\n...,https://www.instagram.com/p/CYOGAc2o5Fn/,32377665,2.185437,"[{'type': 'photo', 'url': 'https://scontent-se...",...,bmw,"{'favoriteCount': 232218, 'commentCount': 695}",232218,695,automobil,Automobil,2.0,True,True,True
9334,2741880971927861036_25749975,Instagram,2022-01-01 22:55:30,2022-05-27 23:40:36,photo,Say hello to renewable adventures.\n\n📸 @lotte...,https://www.instagram.com/p/CYNHjeEKbcs/,34682189,-1.155339,"[{'type': 'photo', 'url': 'https://scontent-se...",...,mercedesbenz,"{'favoriteCount': 77451, 'commentCount': 276}",77451,276,automobil,Automobil,2.0,True,True,True
9335,2741715737230173911_462752227,Instagram,2022-01-01 17:26:46,2022-11-30 12:30:21,album,Commander of the fjords: the Macan gets to kno...,https://www.instagram.com/p/CYMh-_Qs-bX/,26065549,2.848996,"[{'type': 'photo', 'url': 'https://scontent-se...",...,porsche,"{'favoriteCount': 239375, 'commentCount': 462}",239375,462,automobil,Automobil,2.0,True,True,True
9336,2741566902972241322_43109246,Instagram,2022-01-01 12:31:04,2022-07-07 23:43:39,photo,Make this your first decision of 2022 🤔 \nThe ...,https://www.instagram.com/p/CYMAJKjoNWq/,32367896,1.504407,"[{'type': 'photo', 'url': 'https://scontent-se...",...,bmw,"{'favoriteCount': 158901, 'commentCount': 521}",158901,521,automobil,Automobil,2.0,True,True,True


In [50]:
#downloading relevant packages
import nltk
import ssl

try:
    _create_unverified_https_context = ssl._create_unverified_context
except AttributeError:
    pass
else:
    ssl._create_default_https_context = _create_unverified_https_context

nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/theakramer/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/theakramer/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /Users/theakramer/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/theakramer/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

Preprocessing the text data (Tokenization, Lemmatization)

In [51]:
tokenizer = RegexpTokenizer(r'\w+')
posts['post_text'].dropna(inplace=True)
posts['post_text'] = posts['post_text'].str.lower()
posts['post_tokens'] = posts['post_text'].astype(str).apply(tokenizer.tokenize)

# Make a list of english stopwords
stopwords = nltk.corpus.stopwords.words("english")
posts['post_tokens'] = posts['post_tokens'].apply(lambda x: [item for item in x if item not in stopwords])

wordnet_lem = WordNetLemmatizer()

posts['post_lemmas'] = posts['post_tokens'].apply(lambda l: [wordnet_lem.lemmatize(x) for x in l])

In [52]:
words = posts['post_lemmas'].values.tolist() 
words

[['pro', 'green', 'x', 'puma', 'tee', 'professorgreen', 'pumalife'],
 ['day',
  'life',
  'puma',
  'pumalife',
  'morning',
  'green',
  'suede',
  'classic',
  'afternoon',
  'orange',
  'mihara',
  'wedge',
  'evening',
  'navy',
  'suede',
  'classic'],
 ['puma', 'always', 'looking', 'good', 'friend', 'professorgreen'],
 ['mercedes',
  'benz',
  'sl',
  'amg',
  'coupé',
  'black',
  'series',
  'mercedes',
  'mercedesbenz',
  'sl',
  'amg',
  'blackseries',
  'mbcars',
  'thebestornothing',
  'car',
  'car',
  'fuel',
  'consumption',
  'nedc',
  'combined',
  '13',
  '7l',
  '100',
  'km',
  'co2',
  'emission',
  '321g',
  'km'],
 ['say',
  'hello',
  'newest',
  'mercedesamg',
  'powerhouse',
  'sl',
  'amg',
  'coupé',
  'black',
  'series',
  'mercedes',
  'mercedesbenz',
  'sl',
  'amg',
  'blackseries',
  'car',
  'car',
  'mbcars',
  'thebestornothing',
  'fuel',
  'consumption',
  'nedc',
  'combined',
  '13',
  '7l',
  '100',
  'km',
  'co2',
  'emission',
  '321g',
  'k

Train a Gensim word2vec model that represents words as vectors

In [53]:
params = {
    'alpha': 0.05,
    'vector_size': 200, #size
    'window': 10,
    'epochs': 5, #iter
    'min_count': 10,#for craving 30 or 50
    'sample': 1e-4,
    'sg': 1,
    'hs': 0,
    'negative': 10,
}

model = Word2Vec(sentences = words, **params)
wv = model.wv
print("Using trained model", wv)

Using trained model KeyedVectors<vector_size=200, 3835 keys>


Check any word pairs with each other to see how related they are in our specific dataset

In [54]:
check = ['future', 'green', 'work','value'] #change to any words
main_word = 'sustainability' #change to any word
for word in check:
  print(main_word, '<>', word, 100 * round(wv.similarity(main_word, word),2),"%")

sustainability <> future 62.99999952316284 %
sustainability <> green 28.999999165534973 %
sustainability <> work 50.0 %
sustainability <> value 11.999999731779099 %


Define a list of words in the variable `concepts`, a dataframe with the 20 most related words to each word will be shown

In [56]:
concepts = ["sustainable", "sustainability", "green", "future", "climate"] #change to any words

similar_terms = []
print_normal_sims = False
for concept in concepts:
    df = pd.DataFrame()
    vector = wv[concept]
    normal_neighbors = wv.most_similar([vector], topn=30)
    if(print_normal_sims):
        print("\nExact Neighbors (via Gensim)")
    for neighbor in normal_neighbors:
        df = df.append({concept : neighbor[0], concept+'_sim' : round(neighbor[1],3), concept+'_count' : wv.get_vecattr(neighbor[0], "count")}, ignore_index = True)
        if(print_normal_sims):
            print("%-*s  Occ.: %s" % (35,neighbor,wv.get_vecattr(neighbor[0], "count")))
    similar_terms.append(df)  

similarities = pd.concat(similar_terms, axis=1)
similarities

/var/folders/pv/j2wxhdx50y5fbwqrc2yxqwf00000gn/T/ipykernel_66234/2818078530.py:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({concept : neighbor[0], concept+'_sim' : round(neighbor[1],3), concept+'_count' : wv.get_vecattr(neighbor[0], "count")}, ignore_index = True)
/var/folders/pv/j2wxhdx50y5fbwqrc2yxqwf00000gn/T/ipykernel_66234/2818078530.py:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({concept : neighbor[0], concept+'_sim' : round(neighbor[1],3), concept+'_count' : wv.get_vecattr(neighbor[0], "count")}, ignore_index = True)
/var/folders/pv/j2wxhdx50y5fbwqrc2yxqwf00000gn/T/ipykernel_66234/2818078530.py:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({concept : neighb

,sustainable,sustainable_sim,sustainable_count,sustainability,sustainability_sim,sustainability_count,green,green_sim,green_count,future,future_sim,future_count,climate,climate_sim,climate_count
0,sustainable,1.000,1177,sustainability,1.000,1146,green,1.000,690,future,1.000,567,climate,1.000,237
1,sustainability,0.690,1146,sustainablefuture,0.773,15,envy,0.636,14,technological,0.760,10,achieving,0.777,23
2,sustainably,0.664,36,climatepositivity,0.743,12,similar,0.626,13,college,0.757,11,worldenvironmentday,0.766,15
3,socialprogress,0.655,12,socialprogress,0.739,12,totally,0.596,11,innovation,0.744,431,neutrality,0.759,11
4,climatepositivity,0.650,12,strongfoundationboldfuture,0.736,10,machine,0.590,145,smartcities,0.742,13,security,0.756,30
5,environment,0.649,426,climateaction,0.733,34,moon,0.588,11,transformation,0.736,62,protection,0.753,75
6,strongfoundationboldfuture,0.646,10,zerowaste,0.714,10,k12,0.579,11,greenbusiness,0.735,13,fight,0.746,25
7,adhesivetechnologies,0.644,17,sustainabilitymatters,0.711,17,bumbag,0.577,13,pave,0.731,12,importance,0.732,18
8,planet,0.639,84,sap4good,0.697,18,supercar,0.573,22,vidoftheday,0.730,14,tackle,0.725,23
9,investing,0.636,10,greenbusiness,0.695,13,spring,0.571,49,coldplay,0.723,10,target,0.720,61


Uncomment the following if you want to save it

In [ ]:
#similarities.to_csv('similar_terms.csv', header=True, index=False, columns=list(similarities.axes[1]))

In [57]:
print("vocabulary length is:")
print(len(wv))

vocabulary length is:
3835
